In [7]:
import pandas as pd
import requests


In [9]:
# REGIONALES 
# primero 62
# ultimo 16779
df_regionales = pd.read_json('regionales.json')
df_regionales.to_csv('regionales.csv')

In [10]:
# PROVINCIALES
# primero 17
# ultimo 16856
df_provinciales = pd.read_json('provinciales.json')
df_provinciales.to_csv('provinciales.csv')

In [11]:
# DISTRITALES
# primero 19
# ultimo 16860
df_distritales = pd.read_json('distritales.json')
df_distritales.to_csv('distritales.csv')

In [ ]:
# LISTA DE CANDIDATOS - NACIONAL

for id_solicitud_lista in range(16860):
    url = f'https://apiplataformaelectoral.jne.gob.pe/api/v1/candidato/candidatos-lista-internas/{id_solicitud_lista}'
    resp = requests.get(url=url, verify=False)
    data = resp.json()['data']
    if data != []:
        df = pd.DataFrame(data)
        df = df.drop(['archivoBase64'], axis=1)
        df['idSolicitudLista'] = id_solicitud_lista
        df.to_csv(f"listas/{id_solicitud_lista}.csv")